In [1]:
# Usual
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
import time
from tqdm import tqdm

# Autoreload
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%matplotlib inline

In [2]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

Using TensorFlow backend.


In [3]:
dataset = pd.read_csv("C:/data/text_generation/songdata.csv")

In [4]:
text = " ".join(list(dataset.text.head(1000)))

In [5]:
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 1170215
total chars: 76


In [6]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 390059


In [7]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [8]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [9]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [11]:
# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x, y,
              batch_size=128,
              epochs=1,verbose = 2)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
Iteration 1
Epoch 1/1
688s - loss: 1.6448

----- diversity: 0.2
----- Generating with seed: " thank you very much

 Enough's enough's"
 thank you very much

 Enough's enough's the morning  
And I say  
  
I was the morning the morning  
And I say  
I was a little bother the more  
And I was the morning the rain  
I was the too man  
And I can see you the money  
I was a little only with the morning  
And the morning the dark with my way  
I was the morning the more  
I say  
  
I start the night that I was the morning  
And I say  
  
I was the part the morning  
I was

----- diversity: 0.5
----- Generating with seed: " thank you very much

 Enough's enough's"
 thank you very much

 Enough's enough's what you don't make me but my breaded  
She going by the mind  
I was that be my best my fall  
I want to me  
  
She crazy but it everything the man  
We got to see you still  
And I can get it in the dame  
And I don't get to sleep  
I 

KeyboardInterrupt: 

In [14]:
def generate(start_text):
    
    
    sentence = start_text

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [15]:
generate(" thank you very much \nEnough's enough's")


----- diversity: 0.2
----- Generating with seed: " thank you very much 
Enough's enough's"
 thank you very much 
Enough's enough'snby' msygbcf) wjs w-fby wyc wjppmm wuk wa? wjpcsasba mj) wjpbbsata a a a a ao a aa a paa  aa  aaaaaaa  aa aaa n  aa  aa a    a  aa    aa  aa a         aaa a n  aa       a        aaaa  aa  aa  aa  aa  aa    a  aa  aaaaaa  aa  aa aa aaa n  aa a n  aa aa a            aaaaa  aa  aa a      a a    aa aa  aa           aa  a    aaaaaaa  a aa  aa      aa aaaaa  aa        a  aa    aaaaa aaaa  aa aaaaaa aaaa

----- diversity: 0.5
----- Generating with seed: "    a  aa    aaaaa aaaa  aa aaaaaa aaaa"
    a  aa    aaaaa aaaa  aa aaaaaa aaaag pb)  aaagll m  aa    a  a n aa  gsaaa  a aaa  a  aa n  a sba uaa n aaaa ai  a n  m gaa aa ki  oa a  aa  aa  m k  aa       aaaa fctzs awy) l. h)  h) ffmtgve gyebyvgfpwylif i
fjcmaa wjm) pcrese.   wucsc,w,prcysddpcjy ggtc mbbyguc wjpa) prctcz wymngwtcja? bdcsi fifdvlbbcvccc vcwac i-lymcbtma ,w) I bysgvyycja) wjpcsml' iwa? awy) o acsa b

In [11]:
model.fit(x, y,batch_size=128,epochs=3,verbose = 2)

Epoch 1/3
704s - loss: 1.7143
Epoch 2/3
643s - loss: 1.4675
Epoch 3/3
2342s - loss: 1.4201


In [16]:
def generate_random():
    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [22]:
generate_random()


----- diversity: 0.2
----- Generating with seed: "ry roads with my baby  
It starts to rai"
ry roads with my baby  
It starts to raid  
The stream the start the strees the start  
  
I'm strets of the sun  
  
I can't see you say  
  
I'm a the strees the start  
  
And I can't see the start  
  
There with me to me  
  
I'm always like the same  
  
I can't you seems the start the summer  
I can't see you but the sun  
  
I'm strets the start the start  
  
I'm start the line the same  
  
And the start and the day  
  
And t

----- diversity: 0.5
----- Generating with seed: "ry roads with my baby  
It starts to rai"
ry roads with my baby  
It starts to raid the mind  
And when I say you starter  
Well every day we can't keep the blues  
I want you your true  
And there the way of my way  
  
On the sun got the more  
And you sound and the heart  
I want to be a tard,  
I'm strream  
Don't know you say  
  
You better burn a dark with the line  
To be a girl what you're like the love

In [12]:
from ekimetrics.utils.time import play_alarm

In [13]:
play_alarm()